In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-file/tested.csv


# Reading Data

In [2]:
data =  pd.read_csv("/kaggle/input/test-file/tested.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data cleaning

In [3]:
print(data.isnull().sum())
print(data.dtypes)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [4]:
age_mode = data["Age"].mode()[0]
fare_mean = data["Fare"].mean()
print(age_mode)
print(fare_mean)
cleaned = data
cleaned["Age"] = data["Age"].fillna(age_mode)
cleaned["Age"] = cleaned["Age"].astype(int)
cleaned["Fare"] = data["Fare"].fillna(fare_mean)
cleaned["Fare"] = cleaned["Fare"].astype(int)
cleaned = cleaned.drop(['PassengerId','Name','Cabin','Ticket'], axis =1)
print(cleaned.isnull().sum())
print(cleaned.dtypes)

21.0
35.627188489208635
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Survived     int64
Pclass       int64
Sex         object
Age          int64
SibSp        int64
Parch        int64
Fare         int64
Embarked    object
dtype: object


In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
def encode(data):
    cl = labelencoder.fit_transform(data)
    return cl
cleaned["Embarked"] = encode(cleaned["Embarked"])
cleaned["Sex"] = encode(cleaned["Sex"])
cleaned.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,34,0,0,7,1
1,1,3,0,47,1,0,7,2
2,0,2,1,62,0,0,9,1
3,0,3,1,27,0,0,8,2
4,1,3,0,22,1,1,12,2


In [6]:
print(cleaned["Sex"].value_counts())
print(cleaned["Embarked"].value_counts())

1    266
0    152
Name: Sex, dtype: int64
2    270
0    102
1     46
Name: Embarked, dtype: int64


# Split data into 80% train and 20% test

In [7]:
from sklearn.model_selection import train_test_split
X = cleaned.drop(["Survived"] , axis = 1)
Y = cleaned["Survived"]
x_train , x_test , y_train , y_test = train_test_split(X , Y , test_size = 0.2 , random_state = 42)

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc = StandardScaler()
msc = MinMaxScaler()
x_train = msc.fit_transform(x_train)
x_test = msc.fit_transform(x_test)

# Extreme Gradient Boosting

In [9]:
from xgboost import XGBClassifier
gb = XGBClassifier()
gb.fit(x_train , y_train)
yhat = gb.predict(x_test)

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import accuracy_score,confusion_matrix

print("Accuracy score: " , accuracy_score(y_test,yhat))
print("Recall score: " ,recall_score(y_test,yhat, average='micro'))
print("Precision score: " ,precision_score(y_test,yhat , average='micro'))
print("F1 score: " ,f1_score(y_test,yhat , average='micro'))

Accuracy score:  1.0
Recall score:  1.0
Precision score:  1.0
F1 score:  1.0


In [11]:
def pred(data , model):
    data["Sex"] = labelencoder.transform(data["Sex"])
    #data["Embarked"] = labelencoder.transform(data["Embarked"])
    out = model.predict(data)
    if out == 1: 
        return "Survived"
    else:
        return "Not Survived"
di = {}
x = int(input("enter Pclass"))
di["Pclass"] = x
s = input("enter gender")
di["Sex"] = s
x = int(input("enter Age"))
di["Age"] = x
x = int(input("enter SibSp" ))
di["SibSp"] = x
x = int(input("Parch"))
di["Parch"] = x
x = int(input("Fare"))
di["Fare"] = x
s = int(input("Embarked"))
di["Embarked"] = s
df = pd.DataFrame(di , index= [0])


print(f"prediction: {pred(df , gb)}" ) 
    

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.